In [1]:
import pandas as pd
import os

In [2]:
def read_file(filename):
    file = open(filename, mode='rt', encoding='utf-8')
    text = file.read()
    file.close()        
    return text

In [3]:
def to_lines(text):
      sents = text.strip().split('\n')
      sents = [i.split('\t') for i in sents]
      
      return sents

In [4]:
data=read_file("amazon_cells_labelled.txt")

In [5]:
data_lis=to_lines(data)
len(data_lis)

1000

In [6]:
data1=data=read_file("yelp_labelled.txt")

In [7]:
data1_lis=to_lines(data1)
len(data1_lis)

1000

In [8]:
lis=data_lis+data1_lis

In [12]:
import re
def clean_english_sentence(sentence):
    # Remove punctuation using regex
    sentence = re.sub(r'[^\w\s]', '', sentence)
    return sentence.lower()

In [13]:
lis= [[clean_english_sentence(pair[0]), pair[1]] for pair in lis]

In [14]:
lis

[['so there is no way for me to plug it in here in the us unless i go by a converter',
  '0'],
 ['good case excellent value', '1'],
 ['great for the jawbone', '1'],
 ['tied to charger for conversations lasting more than 45 minutesmajor problems',
  '0'],
 ['the mic is great', '1'],
 ['i have to jiggle the plug to get it to line up right to get decent volume',
  '0'],
 ['if you have several dozen or several hundred contacts then imagine the fun of sending each of them one by one',
  '0'],
 ['if you are razr owneryou must have this', '1'],
 ['needless to say i wasted my money', '0'],
 ['what a waste of money and time', '0'],
 ['and the sound quality is great', '1'],
 ['he was very impressed when going from the original battery to the extended battery',
  '1'],
 ['if the two were seperated by a mere 5 ft i started to notice excessive static and garbled sound from the headset',
  '0'],
 ['very good quality though', '1'],
 ['the design is very odd as the ear clip is not very comfortable at 

In [17]:
df = pd.DataFrame(lis, columns=['Sentence', 'Sentiment'])

In [102]:

df['Sentiment'] = pd.to_numeric(df['Sentiment'], errors='coerce').astype(float)

In [24]:

from keras.preprocessing.text import Tokenizer
def tokenization(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

In [103]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Sentence   2000 non-null   object 
 1   Sentiment  2000 non-null   float64
dtypes: float64(1), object(1)
memory usage: 31.4+ KB


In [26]:
eng=df.loc[:,"Sentence"]

In [27]:
eng_tokenizer=tokenization(eng)

In [29]:
eng_tokenizer.word_index

{'the': 1,
 'and': 2,
 'i': 3,
 'a': 4,
 'to': 5,
 'is': 6,
 'it': 7,
 'was': 8,
 'this': 9,
 'of': 10,
 'not': 11,
 'for': 12,
 'my': 13,
 'in': 14,
 'with': 15,
 'very': 16,
 'good': 17,
 'great': 18,
 'phone': 19,
 'that': 20,
 'on': 21,
 'have': 22,
 'you': 23,
 'food': 24,
 'had': 25,
 'place': 26,
 'so': 27,
 'but': 28,
 'service': 29,
 'are': 30,
 'be': 31,
 'we': 32,
 'all': 33,
 'as': 34,
 'at': 35,
 'like': 36,
 'they': 37,
 'time': 38,
 'were': 39,
 'back': 40,
 'one': 41,
 'its': 42,
 'quality': 43,
 'would': 44,
 'really': 45,
 'here': 46,
 'if': 47,
 'dont': 48,
 'from': 49,
 'your': 50,
 'well': 51,
 'just': 52,
 'product': 53,
 'best': 54,
 'will': 55,
 'no': 56,
 'up': 57,
 'an': 58,
 'there': 59,
 'go': 60,
 'has': 61,
 'also': 62,
 'works': 63,
 'ive': 64,
 'only': 65,
 'headset': 66,
 'out': 67,
 'nice': 68,
 'ever': 69,
 'me': 70,
 'battery': 71,
 'im': 72,
 'than': 73,
 'use': 74,
 'or': 75,
 'sound': 76,
 'when': 77,
 'our': 78,
 'get': 79,
 'what': 80,
 'their':

In [30]:
count = 0
for key in eng_tokenizer.word_index:
    count += 1

In [48]:
vocab_length=count+1

In [49]:
vocab_length

3315

In [39]:
eng_l=[]

for pair in lis:
    eng_l.append(len(pair[0].split()))

In [42]:
max(eng_l)

32

In [44]:
from keras.preprocessing.sequence import pad_sequences

def encode_sequences(tokenizer, length, lines):
    # integer encode sequences
    seq = tokenizer.texts_to_sequences(lines)
    # pad sequences with 0 values
    seq = pad_sequences(seq, maxlen=length, padding='post')
    return seq

In [45]:
sequence = encode_sequences(eng_tokenizer,max(eng_l),df.loc[:,"Sentence"])

In [46]:
sequence

array([[ 27,  59,   6, ...,   0,   0,   0],
       [ 17, 108,  86, ...,   0,   0,   0],
       [ 18,  12,   1, ...,   0,   0,   0],
       ...,
       [277,   3,   8, ...,   0,   0,   0],
       [  1, 632, 138, ...,   0,   0,   0],
       [120,  34,  47, ...,   0,   0,   0]])

In [130]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense,Dropout,Layer

In [160]:
import tensorflow as tf
from keras.layers import RepeatVector
embedding_vector_features=40
model=Sequential()
model.add(Embedding(vocab_length,embedding_vector_features,input_length=32))


model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
adam = tf.keras.optimizers.Adam(lr=0.001)
model.compile(optimizer=adam, loss='binary_crossentropy')


C:\Users\tanta\anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [161]:
model.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_15 (Embedding)    (None, 32, 40)            132600    
                                                                 
 lstm_17 (LSTM)              (None, 100)               56400     
                                                                 
 dropout_16 (Dropout)        (None, 100)               0         
                                                                 
 dense_6 (Dense)             (None, 1)                 101       
                                                                 
Total params: 189,101
Trainable params: 189,101
Non-trainable params: 0
_________________________________________________________________


In [162]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(sequence,df.loc[:,"Sentiment"].to_numpy(), test_size=0.33, random_state=42)

In [163]:
model.fit(X_train,y_train,validation_split=0.2,epochs=10,batch_size=64)

Epoch 1/10
17/17 [==============================] - 3s 64ms/step - loss: 0.6940 - val_loss: 0.6924
Epoch 2/10
17/17 [==============================] - 1s 45ms/step - loss: 0.6930 - val_loss: 0.6930
Epoch 3/10
17/17 [==============================] - 1s 45ms/step - loss: 0.6927 - val_loss: 0.6899
Epoch 4/10
17/17 [==============================] - 1s 45ms/step - loss: 0.6323 - val_loss: 0.5771
Epoch 5/10
17/17 [==============================] - 1s 37ms/step - loss: 0.3884 - val_loss: 0.5107
Epoch 6/10
17/17 [==============================] - 1s 42ms/step - loss: 0.1916 - val_loss: 0.4871
Epoch 7/10
17/17 [==============================] - 1s 53ms/step - loss: 0.0938 - val_loss: 0.4187
Epoch 8/10
17/17 [==============================] - 1s 44ms/step - loss: 0.0489 - val_loss: 0.7981
Epoch 9/10
17/17 [==============================] - 1s 43ms/step - loss: 0.0350 - val_loss: 0.5879
Epoch 10/10
17/17 [==============================] - 1s 43ms/step - loss: 0.0247 - val_loss: 0.6206


In [164]:
predict=model.predict(X_test)

In [165]:
import numpy as np
y_pred=np.where(predict > 0.6, 1,0) 

In [166]:
from sklearn.metrics import confusion_matrix

In [167]:
confusion_matrix(y_test,y_pred)

array([[277,  67],
       [ 63, 253]], dtype=int64)

In [168]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.803030303030303

#### Implementing with attention mechanism

In [169]:
from tensorflow.keras import backend as K
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

In [171]:
from tensorflow.keras.layers import Bidirectional
embedding_vector_features=40
model=Sequential()
model.add(Embedding(vocab_length,embedding_vector_features,input_length=32))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(100,return_sequences=True)))
model.add(attention())

model.add(Dense(1,activation='sigmoid'))
adam = tf.keras.optimizers.Adam(lr=0.001)
model.compile(optimizer=adam, loss='binary_crossentropy')

C:\Users\tanta\anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [172]:
model.summary()

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_17 (Embedding)    (None, 32, 40)            132600    
                                                                 
 dropout_18 (Dropout)        (None, 32, 40)            0         
                                                                 
 bidirectional_2 (Bidirectio  (None, 32, 200)          112800    
 nal)                                                            
                                                                 
 attention_5 (attention)     (None, 200)               232       
                                                                 
 dense_7 (Dense)             (None, 1)                 201       
                                                                 
Total params: 245,833
Trainable params: 245,833
Non-trainable params: 0
_______________________________________________

In [173]:
model.fit(X_train,y_train,validation_split=0.2,epochs=20,batch_size=64)

Epoch 1/20
17/17 [==============================] - 4s 84ms/step - loss: 0.6945 - val_loss: 0.6898
Epoch 2/20
17/17 [==============================] - 1s 41ms/step - loss: 0.6910 - val_loss: 0.6880
Epoch 3/20
17/17 [==============================] - 1s 43ms/step - loss: 0.6150 - val_loss: 0.5738
Epoch 4/20
17/17 [==============================] - 1s 42ms/step - loss: 0.3559 - val_loss: 0.5060
Epoch 5/20
17/17 [==============================] - 1s 41ms/step - loss: 0.2040 - val_loss: 0.5534
Epoch 6/20
17/17 [==============================] - 1s 41ms/step - loss: 0.1386 - val_loss: 0.5500
Epoch 7/20
17/17 [==============================] - 1s 41ms/step - loss: 0.0870 - val_loss: 0.7340
Epoch 8/20
17/17 [==============================] - 1s 42ms/step - loss: 0.0581 - val_loss: 0.6678
Epoch 9/20
17/17 [==============================] - 1s 43ms/step - loss: 0.0321 - val_loss: 0.7741
Epoch 10/20
17/17 [==============================] - 1s 45ms/step - loss: 0.0174 - val_loss: 0.8102
Epoch 11/

In [174]:
y_pred=model.predict(X_test)

In [175]:
y_pred=np.where(y_pred > 0.6, 1,0) 

In [176]:
from sklearn.metrics import confusion_matrix

In [177]:
confusion_matrix(y_test,y_pred)

array([[284,  60],
       [ 79, 237]], dtype=int64)

In [178]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.7893939393939394